In [49]:
import pandas as pd
import numpy as np
import pickle


def read_tagging(file_name, lang_choice):
    if lang_choice == 1:
     path = "../data/" + "en" + "/tagging/" + file_name + ".conllu"
    if lang_choice == 2:
     path = "../data/" + "it" + "/tagging/" + file_name + ".conllu"
    if lang_choice == 3:
     path = "../data/" + "es" + "/tagging/" + file_name + ".conllu"
    
    data = pd.read_csv(path, sep="\t", quoting=3, names=["POSITION", "WORD", "TAG"])
    return data



In [50]:
import random

# Funzione di [Sostituzione] per ottenere o generare un pseudonimo per un'entità di tipo PERSON, ORGANIZATION o LOCATION
def get_pseudonym(
    entity_type, entity, pseudonym_counters, pseudonyms,
):

    if entity_type in [
        "PER",
        "ORG",
        "LOC",
        "MISC",
    ]:  # Verifica se l'entità è di uno dei tipi gestiti
        if entity in pseudonyms:  # Verifica se l'entità ha già un pseudonimo assegnato
            return pseudonyms[entity]  # Restituisce l'pseudonimo già assegnato
        else:
            # Se l'entità non ha ancora un pseudonimo, genera uno nuovo
            if entity not in pseudonym_counters[entity_type]:
                # Assegna un numero incrementale come identificativo dell'entità
                pseudonym_counters[entity_type][entity] = (
                    len(pseudonym_counters[entity_type]) + 1
                )
            # Costruisce l'pseudonimo con il formato corretto
            pseudonym = (
                f"[{entity_type.capitalize()}{pseudonym_counters[entity_type][entity]}]"
            )            
            # Memorizza l'pseudonimo per l'entità per evitare duplicazioni
            pseudonyms[entity] = pseudonym
            return pseudonym
    else:
        return entity  # Restituisce l'entità originale se non è di un tipo gestito

# Funzione di Maschera [Parola --> P****a]
def anonymize_mask(entity, tag, entity_counters):
    if tag not in entity_counters:
        entity_counters[tag] = {}

    if entity not in entity_counters[tag]:
        entity_counters[tag][entity] = len(entity_counters[tag]) + 1

    count = entity_counters[tag][entity]

    if len(entity) > 2:
        masked_entity = f"{entity[0]}{'*' * (len(entity) - 2)}{entity[-1]}"
    else:
        masked_entity = "*" * len(entity)

    return f"[{tag.capitalize()}{count}]{masked_entity}"


# Funzione di Lacuna [Parola --> ________]
def anonymize_gap(entity, tag, entity_counters):
    # Inizializziamo il contatore per il tipo di entità se non esiste
    if tag not in entity_counters:
        entity_counters[tag] = {}

    # Incrementiamo il contatore per l'entità specifica se è la prima volta che la incontriamo
    if entity not in entity_counters[tag]:
        entity_counters[tag][entity] = len(entity_counters[tag]) + 1

    # Costruiamo la rappresentazione della lacuna con "_" di dimensione fissa 8
    gap_entity = "_" * 8

    # Restituiamo la stringa formattata con il tag e l'indice dell'entità
    return f"[{tag.capitalize()}{entity_counters[tag][entity]}]{gap_entity}"

def get_user_choice():

    technique_mapping = {
    1: "sostituzione",
    2: "lacuna",
    3: "mascheramento"
    }

    language_mapping ={
    1: "Inglese",
    2: "Italiano",
    3: "Spagnolo"
    }
    
    print("Scegli la Lingua del testo:")
    print("1. inglese")
    print("2. italiano")
    print("3. spagnolo")
    lang_choice = input("Inserisci il numero della tecnica scelta: ")
    int_lang_choice = int(lang_choice)
    print("")
    print("-------------------------------------------")

    print("Scegli la tecnica di anonimizzazione:")
    print("1. Sostituzione")
    print("2. Lacuna")
    print("3. Mascheramento")
    choice = input("Inserisci il numero della tecnica scelta: ")
    int_choice = int(choice)
    print("")

    # Verifica se la scelta è valida
    if int_choice not in technique_mapping:
     raise ValueError("Scelta non valida. Inserisci 1, 2 o 3.")
    
    technique = technique_mapping[int_choice]
    lang = language_mapping[int_lang_choice]
    return int_choice, int_lang_choice, technique, lang

In [51]:

import pandas as pd

def anonymize_document(ner_df, technique):

    # Conversione di tutte le parole a stringhe
    ner_df["WORD"] = ner_df["WORD"].astype(str)

    # Dizionario che tiene traccia del numero di pseudonimi già assegnati per ogni tipo di entità
    pseudonym_counters = {
        "PER": {},  # Contatore per le entità di tipo PERSON
        "ORG": {},  # Contatore per le entità di tipo ORGANIZATION
        "LOC": {},  # Contatore per le entità di tipo LOCATION
        "MISC": {} ,  # Contatore per le entità di tipo MISC
    }

    # Dizionario che mappa gli pseudonimi già assegnati a ciascuna entità
    pseudonyms = {}

    entity_counters = {}

    anonymized_text = ""
    current_tag = None
    buffer = []

    for index, row in ner_df.iterrows():
        word = row["WORD"]
        tag = row["TAG"]

        if tag.startswith("B-"):
            if buffer:
                if current_tag in ["PER", "ORG", "LOC", "MISC"]:

                    if technique == 1:
                        pseudonym = get_pseudonym(
                            current_tag,
                            " ".join(buffer),
                            pseudonym_counters,
                            pseudonyms,
                                                    )
                        anonymized_text += "[" + pseudonym + "]" + " "

                    elif technique == 2:  # Lacuna
                        anonymized_text += (
                            "["
                            + anonymize_gap(
                                " ".join(buffer), current_tag, entity_counters
                            )
                            + "]"
                            + " "
                        )

                    elif technique == 3:  # Mascheramento
                        anonymized_text += (
                            "["
                            + anonymize_mask(
                                " ".join(buffer), current_tag, entity_counters
                            )
                            + "]"
                            + " "
                        )
                else:
                    anonymized_text += " ".join(buffer) + " "

                buffer = []

            current_tag = tag[2:]
            buffer.append(word)

        elif tag.startswith("I-") and current_tag == tag[2:]:
            buffer.append(word)
        else:
            if buffer:
                if current_tag in ["PER", "ORG", "LOC", "MISC"]:
                    if technique == 1:
                        pseudonym = get_pseudonym(
                            current_tag,
                            " ".join(buffer),
                            pseudonym_counters,
                            pseudonyms,
                            
                        )
                        anonymized_text += "[" + pseudonym + "]" + " "

                    elif technique == 2:  # Lacuna
                        anonymized_text += (
                            "["
                            + anonymize_gap(
                                " ".join(buffer), current_tag, entity_counters
                            )
                            + "]"
                            + " "
                        )

                    elif technique == 3:  # Mascheramento
                        anonymized_text += (
                            "["
                            + anonymize_mask(
                                " ".join(buffer), current_tag, entity_counters
                            )
                            + "]"
                            + " "
                        )

                else:
                    anonymized_text += " ".join(buffer) + " "
                buffer = []
                current_tag = None

            if word in [".", ",", "!", "?"]:
                anonymized_text = anonymized_text.rstrip() + word + " "
            else:
                anonymized_text += word + " "

    if buffer:
        if current_tag in ["PER", "ORG", "LOC", "MISC"]:

            if technique == 1:
                pseudonym = get_pseudonym(
                    current_tag,
                    " ".join(buffer),
                    pseudonym_counters,
                    pseudonyms,
                    
                )
                anonymized_text += "[" + pseudonym + "]" + " "

            elif technique == 2:  # Lacuna
                anonymized_text += (
                    "["
                    + anonymize_gap(" ".join(buffer), current_tag, entity_counters)
                    + "]"
                    + " "
                )

            elif technique == 3:  # Mascheramento
                anonymized_text += (
                    "["
                    + anonymize_mask(" ".join(buffer), current_tag, entity_counters)
                    + "]"
                    + " "
                )

       
        else:
            anonymized_text += " ".join(buffer) + " "

    anonymized_text = anonymized_text.strip()

    return anonymized_text

Valutazione delle performance

In [52]:
#conversione di un dataframe che contine le frasi etichettate 
#in un dataframe che contiene solo le entità etichettate nella forma di quadrupe (tag, sentence number, start index, end index)
def extract_entities_from_dataframe(dataframe):
    entity_spans = []

    current_entity_span = None
    current_sentence_index = 0

    for index, row in dataframe.iterrows():
        word = row['WORD']
        tag = row['TAG']
        position = row['POSITION']

        if position == 0:  # Inizio di una nuova frase
            current_sentence_index += 1

        if tag != 'O':
            if tag.startswith('B-'):
                # Se inizia una nuova entità, chiudi quella corrente e inizia una nuova
                if current_entity_span is not None:
                    entity_spans.append(current_entity_span)
                current_entity_span = {'Tag': tag[2:], 'Sentence Number': current_sentence_index}
                current_entity_span['Start Index'] = index
                current_entity_span['End Index'] = index
            elif tag.startswith('I-'):
                # Aggiungi la parola all'entità corrente
                if current_entity_span is not None:
                    current_entity_span['End Index'] = index
            else:
                print("Errore: Tag non riconosciuto.")

        else:
            # Se il tag è "O" ma siamo all'interno di una serie di tag non "O", chiudi l'entità corrente
            if current_entity_span is not None:
                entity_spans.append(current_entity_span)
                current_entity_span = None

    # Aggiungo l'ultima entità se presente
    if current_entity_span is not None:
        entity_spans.append(current_entity_span)

    # Creazione del dataframe di output
    output_data = {'Tag': [], 'Sentence Number': [], 'Start Index': [], 'End Index': []}
    for entity_span in entity_spans:
        output_data['Tag'].append(entity_span['Tag'])
        output_data['Sentence Number'].append(entity_span['Sentence Number'])
        output_data['Start Index'].append(entity_span['Start Index'])
        output_data['End Index'].append(entity_span['End Index'])

    output_df = pd.DataFrame(output_data)

    return output_df

In [53]:
def calculate_accuracy(system_df, golden_df):
    # Unione i due dataframe per confrontare i tag
    merged_df = pd.merge(system_df, golden_df, left_index=True, right_index=True, suffixes=('_system', '_golden'))
    # Conteggio di tag corrispondenti uguali
    correct_tags = (merged_df['TAG_system'] == merged_df['TAG_golden']).sum()
    accuracy = correct_tags / len(system_df)
    accuracy_percent = round(accuracy * 100, 1)
    return accuracy_percent

def calculate_precision_recall(predicted_df, golden_df):
    # Unione delle entità predette e delle entità corrette
    merged_df = pd.merge(predicted_df, golden_df, how='outer', indicator=True)
    # Calcolo dei true positives (TP), false positives (FP) e false negatives (FN)
    TP = merged_df[(merged_df['_merge'] == 'both')].shape[0]
    FP = merged_df[(merged_df['_merge'] == 'right_only')].shape[0]
    FN = merged_df[(merged_df['_merge'] == 'left_only')].shape[0]

    fn_list = merged_df[((merged_df['_merge'] == 'left_only'))].iloc[:, :3]

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    precision_percent = round(precision * 100, 1)
    recall_percent = round(recall * 100, 1)

    return precision_percent, recall_percent, FN,  fn_list


In [56]:

from datetime import datetime

dataset_size = 28000
int_choice, int_lang_choice, technique, language= get_user_choice()
current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

ner_df = read_tagging("viterbi_tag", int_lang_choice)[:dataset_size]
golden_df = read_tagging("golden_tag", int_lang_choice)[:dataset_size]

gold_quadruples_df = extract_entities_from_dataframe(golden_df)
ner_quadruples_df = extract_entities_from_dataframe(ner_df)

precision, recall, nfailed, listfailed = calculate_precision_recall(ner_quadruples_df,gold_quadruples_df)

golden_anonymized_document = anonymize_document(golden_df, int_choice)
vit_anonymized_document = anonymize_document(ner_df, int_choice)
print(vit_anonymized_document)

with open("Results.txt", "a", encoding="utf-8") as file:
 file.write("---HMM Taggin Anonimizzation------"+'\n')
 file.write(current_date+ " | tecnica selezionata: "+ technique + " | Lingua Testo: "+ language )
 file.write('\n')
 file.write('\n')
 file.write(vit_anonymized_document)
 file.write('\n')
 file.write("______________________________________________"+'\n')
 file.write("accuratezza del tagging: "+str(calculate_accuracy(ner_df,golden_df))+"%"+'\n')
 file.write("Precisione del Tagging: "+str(precision)+"%"+'\n')
 file.write("Recall del tagging: " +str(recall)+"%"+'\n')
 file.write("______________________________________________"+'\n')
 file.write('\n')
 file.write("Attenzione, l'anonimizzazione automatica non ha elaborato "+str(nfailed)+" Elementi"+'\n')
 file.write("lista delle entità non elaborate automaticamente:"+'\n')
 file.write(pd.DataFrame.to_string(listfailed))
 file.write('\n')
 file.write('\n')
 file.write("------ Testo anonimizzato d'esempio -------"+'\n')
 file.write(golden_anonymized_document)

 print("Results have been saved to Results.txt")

Scegli la Lingua del testo:
1. inglese
2. italiano
3. spagnolo

-------------------------------------------
Scegli la tecnica di anonimizzazione:
1. Sostituzione
2. Lacuna
3. Mascheramento

On this occasion he failed to gain the support of the [[Loc1]] Miners ' [[Loc2]] and had to stand down. On both these occasions he was backed by the [[Loc1]] Miners ' [[Loc2]], but he was not successful. He also appeared as himself in the 1996 film " [[Per1]] ". The [[Org1]] were created as an expansion franchise in 1993 and [[Loc3]] opened in 1995. He kept busy recording demo tapes at his home and working various jobs, including a position as a contracted security guard at the La [[Loc4]] in [[Loc5]] Jolla. included future [[Org2]] Against the [[Misc1]] and Audioslave drummer [[Per2]] [[Per3]]. Ten " brought the band into the mainstream, and became one of the best selling alternative albums of the 1990 s, being certified 13x Platinum. ] ] Following an intense touring schedule, the band went into th